In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import json

In [3]:
import re

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
    #'Cookie': cookie
}

In [5]:
urlstr = 'https://space.bilibili.com/ajax/member/getSubmitVideos?mid={}&pagesize=30&tid=0&page={}&keyword=&order=pubdate'

In [6]:
uid = 291296062
aids = []
for page in range(1, 2):
    url = urlstr.format(uid, page)
    wb_data = requests.get(url, headers=headers)
    vlist = json.loads(wb_data.text)['data']['vlist']
    aids.extend([video['aid'] for video in vlist])

In [7]:
avurlstr = 'https://www.bilibili.com/video/av{}/'

In [8]:
cids = []
for aid in aids:
    avurl = avurlstr.format(aid)
    wb_data = requests.get(avurl, headers=headers)
    soup = BeautifulSoup(wb_data.text, 'lxml')
    match1 = [cid[5:-1] for cid in re.findall("cid='\d+'", wb_data.text)]
    cids.extend(match1)
    if len(match1) == 0:
        match2 = [cid[4:-1] for cid in re.findall('cid=\d+&', wb_data.text)]
        cids.extend(match2)
    print('#', end='')

#######

In [9]:
def crawl_danmaku(cid):
    url = 'https://comment.bilibili.com/{}.xml'.format(cid)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
    }

    wb_data = requests.get(url, headers=headers)
    soup = BeautifulSoup(wb_data.text, 'lxml')
    danmaku = soup.select('d')
    danmaku_list = [danmaku[i].get_text() for i in range(len(danmaku))]
    
    return danmaku_list

In [10]:
import jieba
import jieba.analyse

In [11]:
def extract_keywords(danmaku_list, top=500):
    seg = [' '.join(jieba.cut(dan)) for dan in danmaku_list]

    sentence = ' '.join(seg)

    tags = jieba.analyse.extract_tags(sentence, top)
    tags_sentence = ' '.join(tags)
    return tags_sentence

In [12]:
danmaku = []
for cid in cids:
    danmaku_list = crawl_danmaku(cid)
    danmaku.extend(danmaku_list)
    print('#', end='')

###############

In [13]:
len(danmaku)

6389

In [14]:
tags_sentence = extract_keywords(danmaku)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\xieyu\AppData\Local\Temp\jieba.cache
Loading model cost 0.900 seconds.
Prefix dict has been built succesfully.


In [15]:
import matplotlib.pyplot as plt
%matplotlib inline

from wordcloud import WordCloud, ImageColorGenerator

In [16]:
def gen_wordcloud(tags_sentence, mask, font, scale=8):
    coloring = plt.imread(mask)
    image_colors = ImageColorGenerator(coloring)
    wordCloud = WordCloud(background_color="white",
                          mask=coloring,
                          font_path=font,
                          random_state=60,
                          max_words=3000,
                          scale=scale).generate(tags_sentence)
    wordCloud.recolor(color_func=image_colors)
    return wordCloud

In [17]:
def save_wordcloud(wordCloud, output):
    wordCloud.to_file(output)

In [18]:
mask = 'hinata.jpg'
font = 'C://Windows/Fonts/msyh.ttc'

In [19]:
wordCloud = gen_wordcloud(tags_sentence, mask, font)
save_wordcloud(wordCloud, 'hinata_cloud.png')

In [20]:
print(len(aids))
print(len(cids))

7
15
